In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
import scipy as sp
import os
import xgboost as xgb
%matplotlib inline

/Users/rezarad/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Let's load the data into pandas dataframes

In [4]:
train = pd.read_csv("./train.csv", parse_dates = ['timestamp'])
print train.shape
test = pd.read_csv("./test.csv", parse_dates = ['timestamp'])
print test.shape
macro = pd.read_csv("./macro.csv", parse_dates = ['timestamp'])
print macro.shape
train.head()

(30471, 292)
(7662, 291)
(2484, 100)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [5]:
macro.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


In [6]:
hftrain = train.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,291]]

In [7]:
hftrain.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo,6.407578e+06,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton,9.589337e+06,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki,4.808270e+06,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino,1.258354e+07,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Basmannoe,8.398461e+06,16331452


In [8]:
hftest = test.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13]]

In [9]:
hftest.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,3.0,Investment,Juzhnoe Butovo,2.615514e+07
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,2.0,Investment,Perovo,9.946335e+06
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,3.0,OwnerOccupier,Poselenie Voskresenskoe,2.149409e+07
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07


In [10]:
house_test.values

NameError: name 'house_test' is not defined

In [ ]:
hftest.shape

In [ ]:
hftrain.describe()

In [ ]:
# check number of missing values per column
np.sum(hftrain.isnull())

Oof. Okay, let's take care of these as we go along. First, look at price.

In [ ]:
plt.figure(figsize=(12,8))
sns.kdeplot(hftrain['price_doc'], shade=True, label='Sale Price')

Skewed way right. Let's log this to normalize.

In [ ]:
plt.figure(figsize=(12,8))
sns.kdeplot(np.log(hftrain['price_doc']), shade=True, label='Sale Price')

Looks better. Forget about dates for now, let's do a correlation plot...maybe we can join some interesting variables from the macro dataset as well?

Look at dates first. are they unique?

In [ ]:
hftrain.groupby("timestamp").agg(['count'])[[1]]

Okay. Let's look at macros dataset, specifically gdp_quart (GDP), deposits_value (Volume of household deposits), deposits_growth (Volume growth of population's deposits), deposits_rate (Average interest rate on deposits), mortgage_value (Volume of mortgage loans), mortgage_growth (Growth of mortgage lending), mortgage_rate (Weighted average rate of mortgage loans), salary, unemployment (Unemployment rate), and employment (Employment rate).

In [ ]:
macro_set = macro.loc[:,['timestamp', 'gdp_quart', 'deposits_value', 'deposits_growth', 'deposits_rate', 'mortgage_value', 
             'mortgage_growth', 'mortgage_rate', 'salary', 'unemployment', 'employment']]

In [ ]:
macro_set.head(10)

In [ ]:
macro_set.shape

Check for duplicates

In [ ]:
np.sum(macro_set.isnull())

See if the timestamps are duplicated.

In [ ]:
macro_set[macro_set.duplicated(['timestamp'], keep=False)]

Negative. Okay, now join with house_train.

In [ ]:
train_full = pd.merge(hftrain, macro_set, how = 'left', on = 'timestamp')

In [ ]:
test_full = pd.merge(hftest, macro_set, how = 'left', on = 'timestamp')

In [ ]:
train_full.shape

In [ ]:
test_full.shape

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
hftrain.head()

Let's look only at hftrain first. Forget about the rest.

In [ ]:
np.sum(hftrain.isnull())

In [ ]:
corrmat = hftrain.corr()
corrmat

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
# Do stuff with the date
def add_date_features(df):
    '''(DataFrame) -> DataFrame
    
    Will add some specific columns based on the date
    of the sale.
    '''
    #Convert to datetime to make extraction easier
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    #Extract features
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['year'] = df['timestamp'].dt.year
    
    #These features inspired by Bruno's Notebook at https://www.kaggle.com/bguberfain/naive-xgb-lb-0-317
    #Week-Year
    week_year = df['timestamp'].dt.weekofyear + df['timestamp'].dt.year * 100
    week_year_map = week_year.value_counts().to_dict()
    df['week_year'] = week_year.map(week_year_map)
    #df.drop('timestamp', axis=1, inplace=True)
    return df

In [ ]:
hftrain = add_date_features(hftrain)

In [ ]:
train_df.apply(lambda x: type(x[0]))

In [ ]:
hftrain.head()

In [ ]:
sns.boxplot(x='material', 
            y='price_doc', 
            data=hftrain, 
            palette='Set3')

In [ ]:
print(hftrain['material'].value_counts(dropna=False))
print(hftest['material'].value_counts(dropna=False))

Nothing missing in the test, so let's look into how to deal with NAs in training.

In [ ]:
hftrain.loc[hftrain['material'] == 3]
hftest.loc[hftest['material'] == 3]

# 3's seem to be negligible, let's replace them with 0
hftrain.loc[train['material'] == 3, 'material'] = np.nan
hftest.loc[test['material'] == 3, 'material'] = np.nan

In [ ]:
# Turn that into a function
def clean_materials(df):
    df.loc[df['material'] ==3, 'material'] = np.nan
    return df

In [ ]:
sns.boxplot(x='material', 
            y='price_doc', 
            data=hftrain, 
            palette='Set3')

Let's look at state

In [ ]:
print(hftrain['state'].value_counts(dropna=False))
print(hftest['state'].value_counts(dropna=False))

In [ ]:
# What's up with that 33 in training? Maybe they meant 3. We;ll change that.
hftrain.loc[hftrain['state'] == 33, 'state'] = 3

In [ ]:
# Let's look at product type
sns.boxplot(x='product_type', 
            y='price_doc', 
            data=hftrain, 
            palette='Set3')

In [ ]:
# check missingness
print(hftrain['product_type'].value_counts(dropna=False))
print(hftest['product_type'].value_counts(dropna=False))
# looks okay for the most part

In [ ]:
## floor / max_floor
hftrain.plot.scatter(x='floor', y='max_floor')

By definition, floor (which the apartment is on) cannot be higher than the max floor of the building. We're seeing a whole lot of points on the bottom there with max_floor equal to zero, and some crazy big outliers which we might want to take a look at.

In [ ]:
# First the outliers. Which building have greater than 60 floors?
## max_floor
hftrain.loc[hftrain['max_floor'] > 60]

We can't really make any inferences on the max_floors of 99, but the 117 is probably an error. We'll change that to 17, and make the other ones NaN.

In [ ]:
hftrain.loc[hftrain['max_floor'] == 117, 'max_floor'] = 17
hftrain.loc[hftrain['max_floor'] > 60, 'max_floor'] = np.nan

In [ ]:
# Investigate floors greater than 60
hftrain.loc[hftrain['floor'] > 60]

In [ ]:
# Seems like an error. Change it to 7.

In [ ]:
hftrain.loc[hftrain['floor'] == 77, 'floor'] = 7

In [ ]:
hftrain.plot.scatter(x='floor', y='max_floor')

In [ ]:
# Looks better, might still want to do something about that bottom portion.

In [ ]:
# Build year
hftrain['build_year'].value_counts(dropna=False).sort_index()
hftest['build_year'].value_counts(dropna=False).sort_index()
# note: there are some build years in 2017-2019 in test set. Might not be by accident

In [ ]:
# 20052009 is weird. Make that 2007. Anything less than year 215 is weird too. maybe we can make those the median...
# or missing like Shu did
hftrain.loc[hftrain['build_year'] == 20052009, 'build_year'] = 2007
hftrain.loc[hftrain['build_year'] == 4965, 'build_year'] = np.nan
hftrain.loc[hftrain['build_year'] < 1800, 'build_year'] = np.nan
hftest.loc[hftest['build_year'] < 1800, 'build_year'] = np.nan

In [ ]:
# Number of rooms vs full_sq
hftrain.plot.scatter(x='num_room', y='full_sq')

In [ ]:
# Number of rooms vs full_sq
hftest.plot.scatter(x='num_room', y='full_sq')

In [ ]:
# Sup with 15 rooms and above?
hftrain.loc[hftrain['num_room'] > 15]
# Let's make them go bye-bye

In [ ]:
hftrain.loc[hftrain['num_room'] > 15, 'num_room'] = np.nan
hftest.loc[hftest['num_room'] > 15, 'num_room'] = np.nan

In [ ]:
# Let's create a new variable called building age. 2017 - build_year
hftrain['building_age'] = 2017 - hftrain['build_year']
hftest['building_age'] = 2017 - hftest['build_year']

In [ ]:
hftrain.loc[hftrain['building_age'] < 0, 'building_age'] = np.nan

In [ ]:
# How bout we see building age versus square footage
hftrain.plot.scatter(x='building_age', y='full_sq')
# no relationship it seems

In [ ]:
# how bout building age with price?
hftrain.plot.scatter(x='building_age', y='price_doc')
# slightly bimodal. newer buildings are more expensive.

In [ ]:
# Let's look at full_sq
sns.kdeplot(hftrain['full_sq'], shade=True, label='Square Footage')

In [ ]:
sns.kdeplot(hftest['full_sq'], shade=True, label='Square Footage')

In [ ]:
hftrain.loc[hftrain['full_sq'] > 1000] # remove this

In [ ]:
hftest.loc[hftest['full_sq'] > 500] # remove this

In [ ]:
hftrain.loc[hftrain['full_sq'] > 1000, 'full_sq'] = np.nan

In [ ]:
# Maybe we can logify that full_sq
sns.kdeplot(np.log(hftrain['full_sq']), shade=True, label='Square Footage')

In [ ]:
# How about number of rooms vs price?
hftrain.plot.scatter(x='num_room', y='price_doc')

In [ ]:
# Add year-month and plot over time
hftrain["yearmonth"] = hftrain["timestamp"].dt.year*100 + hftrain["timestamp"].dt.month
hftest["yearmonth"] = hftest["timestamp"].dt.year*100 + hftest["timestamp"].dt.month

In [ ]:
Plot price over time

In [ ]:
plt.figure(figsize=(12,8))
sns.pointplot(x='yearmonth', y='price_doc', data=hftrain)
plt.ylabel('price_doc', fontsize=12)
plt.xlabel('yearmonth', fontsize=12)
plt.title('Price distribution by year and week_num')
plt.xticks(rotation='vertical')
plt.show()
# We can see in macro that price drops significantly after.

In [ ]:
# Let's investigate subarea. How many values of each?
hftrain['sub_area'].value_counts(dropna=False).sort_index()

In [ ]:
sub_area_group = hftrain.groupby(['sub_area']).price_doc.agg(['count', 'sum', 'min', 'max', 'mean', 'std', 'median'])

In [ ]:
sub_area_group

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='sub_area', y='price_doc', data=hftrain, order = sub_area_group)
plt.ylabel('price_doc', fontsize=12)
plt.xlabel('sub_area', fontsize=12)
plt.title('Price distribution by sub_area')
plt.xticks(rotation='vertical')
plt.show()




In [2]:
import missingno as msno

msno.missingno.matrix(hftrain.loc[:, features[0:20]])

NameError: name 'hftrain' is not defined